In [21]:
import gym
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import copy
import time
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
#env =FrozenLakeEnv(is_slippery=False,map_name="8x8")
env.render()


SFFF
FHFH
FFFH
HFFG


In [23]:
class Net(nn.Module):
    def __init__(self,para,bias=True):
        super(Net,self).__init__()
        self.input_dim=para['input_dim']
        self.output_dim=para['output_dim']
        self.hidden_dim=para['hidden_dim']
        
        self.w1=Parameter(torch.FloatTensor(self.input_dim,self.hidden_dim))
        self.b1=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w1)
#        nn.init.kaiming_uniform_(self.w1)
        nn.init.uniform_(self.b1,-0.001,0.001)

        self.w2=Parameter(torch.FloatTensor(self.hidden_dim,self.hidden_dim))
        self.b2=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w2)
#        nn.init.kaiming_uniform_(self.w2)
        nn.init.uniform_(self.b2,-0.001,0.001)
        
        self.w3=Parameter(torch.FloatTensor(self.hidden_dim,self.hidden_dim))
        self.b3=Parameter(torch.FloatTensor(self.hidden_dim))
        nn.init.kaiming_normal_(self.w3)
#        nn.init.kaiming_uniform_(self.w3)
        nn.init.uniform_(self.b3,-0.001,0.001)
        
        self.w4=Parameter(torch.FloatTensor(self.hidden_dim,self.output_dim))
        self.b4=Parameter(torch.FloatTensor(self.output_dim))
        nn.init.kaiming_normal_(self.w4)
#        nn.init.kaiming_uniform_(self.w4)
        nn.init.uniform_(self.b4,-0.001,0.001)
        
    def forward(self,X):
        
        H=torch.relu( torch.einsum('li,ij->lj',X,self.w1)+self.b1)
        H=torch.relu( torch.einsum('li,ij->lj',H,self.w2)+self.b2)
        H=torch.relu( torch.einsum('li,ij->lj',H,self.w3)+self.b3)
        Y=torch.einsum('li,ij->lj',H,self.w4)+self.b4

        return Y

    

In [24]:
input_dim=env.observation_space.n
output_dim=env.action_space.n
hidden_dim=100

para={'input_dim': input_dim, 'output_dim': output_dim, 'hidden_dim': hidden_dim}
model=Net(para)
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr)

In [25]:
max_episodes=300
max_step=100
lr=0.001
lr2=1.0
gamma=0.99
epsilon0 = 1.0


# initialize

epsilon=epsilon0
min_step=max_step
min_step_episode=max_episodes
min_step_path=[]
for episode in range(max_episodes):
    epsilon=epsilon*0.99
    state = env.reset()
    total_reward = 0
    state_list = [state]
    win=0

    for step in range(0,max_step):
        state_m=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m[0][state]=1.0
        state_m_torch=Variable(torch.from_numpy(state_m))
        Q_pred=model(state_m_torch)
        Q_pred=Q_pred.cpu().detach().numpy()
        if np.random.rand()>epsilon :
            action=np.argmax(Q_pred[0])
        else:
            action=np.random.choice(env.action_space.n)

        state_next,reward,game_over,_ = env.step(action)
        
        if reward==0 : 
            reward=-0.01
        if reward!=1.0 and game_over==1:
            reward=-1.0
            
        state_m_next=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m_next[0][state_next]=1.0
    
        state_m_next_torch=Variable(torch.from_numpy(state_m_next))
        Q_pred_next=model(state_m_next_torch)
        Q_pred_next=Q_pred_next.cpu().detach().numpy()


        if np.random.rand()>epsilon :
            action_next=np.argmax(Q_pred_next[0])
        else:
            action_next=np.random.choice(env.action_space.n)

        Y_true=copy.deepcopy(Q_pred[0])

        if reward==1.0 and game_over==1:
            Y_true[action]= reward
        else :
            Y_true[action]=reward+gamma* (Q_pred_next[0][action_next])
        Y_true=np.reshape(Y_true,[1,env.action_space.n])
        for j in range(0,1):
            optimizer.zero_grad()
            state_m_torch=Variable(torch.from_numpy(state_m))
            Y_pred=model(state_m_torch)
            Y_true_torch=torch.from_numpy(Y_true)
#            print(Y_pred,Y_true_torch)
#            print(Y_pred.shape,Y_true_torch.shape)
            loss=criterion(Y_pred,Y_true_torch)
            loss.backward(retain_graph=True)
            optimizer.step()
            cost=loss.data
#            print(cost)
            

        state=state_next

        total_reward=total_reward+reward
        state_list.append(state)
        if game_over and reward==1:
            win=1
            if step< min_step:
                min_step=step
                min_step_path=state_list
                min_step_episode=episode
        if game_over==1:
            break


    line_out="Episode: %d, step: %d, total_reward: %f, epsilon: %f" %(episode, step, total_reward,epsilon)
    print(line_out)
    if reward==1 :
        print(state_list)
print( "min step: ", min_step, "min step episode", min_step_episode)
print("shortest path: ", min_step_path)


Episode: 0, step: 6, total_reward: -1.060000, epsilon: 0.990000
Episode: 1, step: 2, total_reward: -1.020000, epsilon: 0.980100
Episode: 2, step: 3, total_reward: -1.030000, epsilon: 0.970299
Episode: 3, step: 9, total_reward: -1.090000, epsilon: 0.960596
Episode: 4, step: 17, total_reward: -1.170000, epsilon: 0.950990
Episode: 5, step: 1, total_reward: -1.010000, epsilon: 0.941480
Episode: 6, step: 3, total_reward: -1.030000, epsilon: 0.932065
Episode: 7, step: 27, total_reward: -1.270000, epsilon: 0.922745
Episode: 8, step: 16, total_reward: -1.160000, epsilon: 0.913517
Episode: 9, step: 7, total_reward: -1.070000, epsilon: 0.904382
Episode: 10, step: 2, total_reward: -1.020000, epsilon: 0.895338
Episode: 11, step: 2, total_reward: -1.020000, epsilon: 0.886385
Episode: 12, step: 13, total_reward: -1.130000, epsilon: 0.877521
Episode: 13, step: 4, total_reward: -1.040000, epsilon: 0.868746
Episode: 14, step: 1, total_reward: -1.010000, epsilon: 0.860058
Episode: 15, step: 2, total_rew

Episode: 114, step: 7, total_reward: 0.930000, epsilon: 0.314809
[0, 0, 1, 2, 2, 6, 10, 14, 15]
Episode: 115, step: 5, total_reward: 0.950000, epsilon: 0.311661
[0, 1, 2, 6, 10, 14, 15]
Episode: 116, step: 3, total_reward: -1.030000, epsilon: 0.308544
Episode: 117, step: 5, total_reward: -1.050000, epsilon: 0.305459
Episode: 118, step: 5, total_reward: 0.950000, epsilon: 0.302404
[0, 1, 2, 6, 10, 14, 15]
Episode: 119, step: 6, total_reward: -1.060000, epsilon: 0.299380
Episode: 120, step: 7, total_reward: -1.070000, epsilon: 0.296387
Episode: 121, step: 9, total_reward: 0.910000, epsilon: 0.293423
[0, 1, 2, 6, 10, 6, 10, 14, 10, 14, 15]
Episode: 122, step: 37, total_reward: 0.630000, epsilon: 0.290488
[0, 1, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 1, 2, 6, 2, 2, 6, 2, 6, 2, 6, 10, 14, 15]
Episode: 123, step: 18, total_reward: 0.820000, epsilon: 0.287584
[0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 2, 3, 2, 2, 3, 2, 6, 10, 14, 15]
Episode: 124, step: 4, total_reward: -

Episode: 205, step: 44, total_reward: -1.440000, epsilon: 0.126139
Episode: 206, step: 7, total_reward: 0.930000, epsilon: 0.124878
[0, 4, 0, 4, 8, 9, 13, 14, 15]
Episode: 207, step: 44, total_reward: -1.440000, epsilon: 0.123629
Episode: 208, step: 1, total_reward: -1.010000, epsilon: 0.122393
Episode: 209, step: 90, total_reward: 0.100000, epsilon: 0.121169
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 8, 9, 10, 14, 15]
Episode: 210, step: 5, total_reward: 0.950000, epsilon: 0.119957
[0, 4, 8, 9, 10, 14, 15]
Episode: 211, step: 7, total_reward: 0.930000, epsilon: 0.118758
[0, 4, 8, 9, 13, 14, 10, 14, 15]
Episode: 212, step: 5, total_reward: 0.950000, epsilon: 0.117570
[0, 4, 8, 9, 13, 14, 15]
Episode: 213, step: 5, total_reward: 0.950000, epsilon: 0.116394
[0, 4, 8, 9, 13

Episode: 297, step: 5, total_reward: 0.950000, epsilon: 0.050037
[0, 4, 8, 9, 10, 14, 15]
Episode: 298, step: 5, total_reward: 0.950000, epsilon: 0.049536
[0, 4, 8, 9, 10, 14, 15]
Episode: 299, step: 5, total_reward: 0.950000, epsilon: 0.049041
[0, 4, 8, 9, 10, 14, 15]
min step:  5 min step episode 111
shortest path:  [0, 1, 2, 6, 10, 14, 15]


In [19]:
#state=np.arange(0,15)
#sess.run(init)
state_m=np.zeros([input_dim,env.observation_space.n],dtype=np.float32)
for i in range(0,input_dim):
    state_m[i][i]=1.0
print(state_m)
state_m_torch=Variable(torch.from_numpy(state_m))
Q_pred=model(state_m_torch)
Q_pred=Q_pred.cpu().detach().numpy()
print(Q_pred)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[-1.0459996  -1.0604872  -1.0491861  -1.0497097 ]
 [-1.0421464  -1.086665   -1.0567379  -1.0561626 ]
 [-1.0516323  -1.1133238  -1.0788728  -1.0608284 ]
 [-1.0758016  -1.1443727  -1.0994091  -1.0534861 ]
 [-1.0638142  -1.1552012  -1.1362579  -1.0982205 ]
 [-1.0353754  -1.3108914  -1.1206791  -1.0270288 ]
 [-1.0922614  -1.2472193  -1.1703107  -1.0898064 ]
 [-1.0888352  -1.2299815  -1.1160078  -1.0590338 ]
 [-1.037973   -1.0792972  -1.0525484  -1.0419304 ]
 [-1.0615772  -1.1182106  -1.0807985  -1.0442165 ]
 [-1.0476023  -1.1782326  -1.1085908  -1.0214463 ]
 [-0.9677133  -1.8786765  -1.2827076  -1.0443547 ]
 [-1.0474529  -1.2051544  -1.2002698  -1.0542508 ]
 [-1.0881368  -1.3156201  -1.1192148  -1.0242763 ]
 [-1.1503674  -1.2902833  -1.2034274  -1.1039988 ]
 [-1.101126   -1.2671618  -1.1716375  -1.0363016 ]
 [-1.0636882  -1.2337697  -1